In [2]:
%cd /content/drive/MyDrive/job/recommender_models/components/lightgbm_ranker

/content/drive/MyDrive/job/recommender_models/components/lightgbm_ranker


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb

In [108]:
train_data_path = '../../data/train.pkl'
valid_data_path = '../../data/valid.pkl'
feature_cols = ['price', 'sales_channel_id']
sample_weight_col = 'buy'
target_col = 'buy'
index_cols = ['customer_id']

In [121]:
train = pd.read_pickle(train_data_path)
valid = pd.read_pickle(valid_data_path)
train_group=train.groupby(index_cols)[index_cols[0]].count().to_numpy()
valid_group=valid.groupby(index_cols)[index_cols[0]].count().to_numpy()
sample_weight=train[sample_weight_col]

In [122]:
params = {
    'objective': "lambdarank",
    'boosting_type': "gbdt",
    'metric': "ndcg",
    'num_leaves': 2**7-1,
    'max_depth': 7,
    'num_boost_round': 100,
    'early_stopping_rounds': 10,
    'verbosity': 10,
    'random_state': 42,
}

In [134]:
model = lgb.LGBMRanker(
    objective="lambdarank",
    boosting_type='gbdt', 
    metric='ndcg',
    num_leaves=2**7-1,
    max_depth=7, 
    learning_rate=0.1, 
    n_estimators=100,
    early_stopping_rounds=10,
    random_state=42,
)

In [136]:
model.fit(
    X=train[feature_cols],
    y=train[target_col],
    sample_weight=sample_weight,
    group=train_group,
    eval_set=[(valid[feature_cols], valid[target_col])],
    eval_names=['valid'],
    eval_at=2,
    eval_group=[valid_group],
    )

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid's ndcg@1: 0.699045
Training until validation scores don't improve for 10 rounds.
[2]	valid's ndcg@1: 0.699045
[3]	valid's ndcg@1: 0.699045
[4]	valid's ndcg@1: 0.699045
[5]	valid's ndcg@1: 0.699045
[6]	valid's ndcg@1: 0.699045
[7]	valid's ndcg@1: 0.699045
[8]	valid's ndcg@1: 0.699045
[9]	valid's ndcg@1: 0.699045
[10]	valid's ndcg@1: 0.699045
[11]	valid's ndcg@1: 0.699045
Early stopping, best iteration is:
[1]	valid's ndcg@1: 0.699045


LGBMRanker(early_stopping_rounds=10, eval_at=2, max_depth=7, metric='ndcg',
           num_leaves=127, objective='lambdarank', random_state=42)

In [82]:
model.best_score_

defaultdict(dict, {'valid': {'ndcg@2': 0.7517626488567579}})

(798269, 6)